In [1]:
import os
import shutil
from collections import Counter
import numpy as np
import torch
import torch.utils.data
import pandas as pd
import argparse
import re
from helpers import makedir
import model
import save
from log import create_logger
import train_and_test as tnt
from helpers import makedir
import find_nearest
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import push

from preprocess import mean, std, preprocess_input_function
from settings import train_dir, test_dir, train_push_dir
from settings import img_size, num_classes, prototype_activation_function, add_on_layers_type

from bounding_box_metrics import bounding_box_overlap
from find_nearest import find_k_nearest_patches_to_prototypes
from settings import coefs



In [8]:
'''
Model initialization for vgg base; do not run this block when creating other models 
'''

base_architecture = 'vgg19'
prototype_shape = (2000, 128, 1, 1)

ppnet = model.construct_PPNet(base_architecture=base_architecture,
                              pretrained=True, img_size=img_size,
                              prototype_shape=prototype_shape,
                              num_classes=num_classes,
                              prototype_activation_function=prototype_activation_function,
                              add_on_layers_type=add_on_layers_type)

ppnet = ppnet.cuda()
ppnet_multi = torch.nn.DataParallel(ppnet)
ppnet_multi = torch.nn.DataParallel(ppnet).cuda()
class_specific = True

warm_optimizer_lrs = {'add_on_layers': 1e-3,
                      'prototype_vectors': 1e-3}
warm_optimizer_specs = \
    [{'params': ppnet.add_on_layers.parameters(), 'lr': warm_optimizer_lrs['add_on_layers'], 'weight_decay': 1e-3},
     {'params': ppnet.prototype_vectors, 'lr': warm_optimizer_lrs['prototype_vectors']},
    ]
warm_optimizer = torch.optim.Adam(warm_optimizer_specs)

joint_optimizer_lrs = {'features': 1e-4,
                       'add_on_layers': 1e-3,
                       'prototype_vectors': 1e-3}
joint_optimizer_specs = \
[{'params': ppnet_multi.module.features.parameters(), 'lr': joint_optimizer_lrs['features'], 'weight_decay': 1e-3}, # bias are now also being regularized
 {'params': ppnet_multi.module.add_on_layers.parameters(), 'lr': joint_optimizer_lrs['add_on_layers'], 'weight_decay': 1e-3},
 {'params': ppnet_multi.module.prototype_vectors, 'lr': joint_optimizer_lrs['prototype_vectors']},
]
joint_optimizer = torch.optim.Adam(joint_optimizer_specs)

joint_lr_step_size = 5
gamma = 0.5
joint_lr_scheduler = torch.optim.lr_scheduler.StepLR(joint_optimizer, step_size=joint_lr_step_size, gamma=0.5)

last_layer_optimizer_lr = 1e-4
last_layer_optimizer_specs = [{'params': ppnet.last_layer.parameters(), 'lr': last_layer_optimizer_lr}]
last_layer_optimizer = torch.optim.Adam(last_layer_optimizer_specs)

train_batch_size, test_batch_size, train_push_batch_size = 128, 128, 128
push_epochs = [20, 40, 60, 99]
push_saved_epochs = [20, 40, 60, 99]

model_dir = './ppnet_results/007_vgg19/'
makedir(model_dir)
log, logclose = create_logger(log_filename=os.path.join(model_dir, 'train.log'))

In [4]:
'''
Model initialization for resnet base; do not run this block when creating other models 
'''

base_architecture = 'resnet34'
prototype_shape = (2000, 128, 1, 1)

ppnet = model.construct_PPNet(base_architecture=base_architecture,
                              pretrained=True, img_size=img_size,
                              prototype_shape=prototype_shape,
                              num_classes=num_classes,
                              prototype_activation_function=prototype_activation_function,
                              add_on_layers_type=add_on_layers_type)

ppnet = ppnet.cuda()
ppnet_multi = torch.nn.DataParallel(ppnet)
ppnet_multi = torch.nn.DataParallel(ppnet).cuda()
class_specific = True

warm_optimizer_lrs = {'add_on_layers': 1e-3,
                      'prototype_vectors': 1e-3}
warm_optimizer_specs = \
    [{'params': ppnet.add_on_layers.parameters(), 'lr': warm_optimizer_lrs['add_on_layers'], 'weight_decay': 1e-3},
     {'params': ppnet.prototype_vectors, 'lr': warm_optimizer_lrs['prototype_vectors']},
    ]
warm_optimizer = torch.optim.Adam(warm_optimizer_specs)

joint_optimizer_lrs = {'features': 1e-4,
                       'add_on_layers': 1e-3,
                       'prototype_vectors': 1e-3}
joint_optimizer_specs = \
[{'params': ppnet_multi.module.features.parameters(), 'lr': joint_optimizer_lrs['features'], 'weight_decay': 1e-3}, # bias are now also being regularized
 {'params': ppnet_multi.module.add_on_layers.parameters(), 'lr': joint_optimizer_lrs['add_on_layers'], 'weight_decay': 1e-3},
 {'params': ppnet_multi.module.prototype_vectors, 'lr': joint_optimizer_lrs['prototype_vectors']},
]
joint_optimizer = torch.optim.Adam(joint_optimizer_specs)

joint_lr_step_size = 5
gamma = 0.5
joint_lr_scheduler = torch.optim.lr_scheduler.StepLR(joint_optimizer, step_size=joint_lr_step_size, gamma=0.5)

last_layer_optimizer_lr = 1e-4
last_layer_optimizer_specs = [{'params': ppnet.last_layer.parameters(), 'lr': last_layer_optimizer_lr}]
last_layer_optimizer = torch.optim.Adam(last_layer_optimizer_specs)

train_batch_size, test_batch_size, train_push_batch_size = 128, 128, 128
push_epochs = [20, 40, 60, 99]
push_saved_epochs = [20, 40, 60, 99]

model_dir = '/scratch/users/jiaxun1218/saved_models/resnet34/'
makedir(model_dir)
log, logclose = create_logger(log_filename=os.path.join(model_dir, 'train.log'))


In [10]:
'''
Model initialization for densenet; do not run this block when creating other models 
'''

base_architecture = 'densenet161'
prototype_shape = (2000, 128, 1, 1)

ppnet = model.construct_PPNet(base_architecture=base_architecture,
                              pretrained=True, img_size=img_size,
                              prototype_shape=prototype_shape,
                              num_classes=num_classes,
                              prototype_activation_function=prototype_activation_function,
                              add_on_layers_type=add_on_layers_type)

ppnet = ppnet.cuda()
ppnet_multi = torch.nn.DataParallel(ppnet)
ppnet_multi = torch.nn.DataParallel(ppnet).cuda()
class_specific = True

warm_optimizer_lrs = {'add_on_layers': 1e-3,
                      'prototype_vectors': 1e-3}
warm_optimizer_specs = \
    [{'params': ppnet.add_on_layers.parameters(), 'lr': warm_optimizer_lrs['add_on_layers'], 'weight_decay': 1e-3},
     {'params': ppnet.prototype_vectors, 'lr': warm_optimizer_lrs['prototype_vectors']},
    ]
warm_optimizer = torch.optim.Adam(warm_optimizer_specs)

joint_optimizer_lrs = {'features': 1e-4,
                       'add_on_layers': 1e-3,
                       'prototype_vectors': 1e-3}
joint_optimizer_specs = \
[{'params': ppnet_multi.module.features.parameters(), 'lr': joint_optimizer_lrs['features'], 'weight_decay': 1e-3}, # bias are now also being regularized
 {'params': ppnet_multi.module.add_on_layers.parameters(), 'lr': joint_optimizer_lrs['add_on_layers'], 'weight_decay': 1e-3},
 {'params': ppnet_multi.module.prototype_vectors, 'lr': joint_optimizer_lrs['prototype_vectors']},
]
joint_optimizer = torch.optim.Adam(joint_optimizer_specs)

joint_lr_step_size = 5
gamma = 0.5
joint_lr_scheduler = torch.optim.lr_scheduler.StepLR(joint_optimizer, step_size=joint_lr_step_size, gamma=0.5)

last_layer_optimizer_lr = 1e-4
last_layer_optimizer_specs = [{'params': ppnet.last_layer.parameters(), 'lr': last_layer_optimizer_lr}]
last_layer_optimizer = torch.optim.Adam(last_layer_optimizer_specs)

train_batch_size, test_batch_size, train_push_batch_size = 128, 128, 128
push_epochs = [20, 40, 60, 99]
push_saved_epochs = [20, 40, 60, 99]

model_dir = '/scratch/users/jiaxun1218/saved_models/densenet161/'
makedir(model_dir)
log, logclose = create_logger(log_filename=os.path.join(model_dir, 'train.log'))


Downloading: "https://download.pytorch.org/models/densenet161-8d451a50.pth" to ./pretrained_models/densenet161-8d451a50.pth
100%|██████████| 110M/110M [00:01<00:00, 116MB/s] 


In [11]:
normalize = transforms.Normalize(mean=mean,
                                 std=std)
train_dataset = datasets.ImageFolder(
        train_dir,
        transforms.Compose([
        transforms.Resize(size=(img_size, img_size)),
        transforms.ToTensor(),
        normalize,
]))

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=train_batch_size, shuffle=True,
    num_workers=2, pin_memory=False)

# push set
train_push_dataset = datasets.ImageFolder(
    train_push_dir,
    transforms.Compose([
        transforms.Resize(size=(img_size, img_size)),
        transforms.ToTensor(),
]))

train_push_loader = torch.utils.data.DataLoader(
    train_push_dataset, batch_size=train_push_batch_size, shuffle=False,
    num_workers=2, pin_memory=False)

# test set
test_dataset = datasets.ImageFolder(
    test_dir,
    transforms.Compose([
        transforms.Resize(size=(img_size, img_size)),
        transforms.ToTensor(),
        normalize,
]))

test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=test_batch_size, shuffle=False,
    num_workers=2, pin_memory=False)

In [12]:
for epoch in range(0, 100):
    print('epoch: \t{0}'.format(epoch))

    #if epoch in range(0, 10):
    #    tnt.joint_warm(model=ppnet_multi, log=log)
    #else:
    
    if epoch in range(0, 5):
        tnt.warm_only(model=ppnet_multi, log=log)
        _ = tnt.train(model=ppnet_multi, dataloader=train_loader, optimizer=warm_optimizer,
                  class_specific=class_specific, coefs=coefs, log=log)
        
    else:
        tnt.joint(model=ppnet_multi, log=log)
        _ = tnt.train(model=ppnet_multi, dataloader=train_loader, optimizer=joint_optimizer,
                  class_specific=class_specific, coefs=coefs, log=log)
        joint_lr_scheduler.step()

#     accu = tnt.test(model=ppnet_multi, dataloader=test_loader,
#                     class_specific=class_specific, log=log)
#     save.save_model_w_condition(model=ppnet, model_dir=model_dir, model_name=str(epoch) + 'nopush', accu=accu,
#                                 target_accu=0.70, log=log)

    if epoch in push_epochs:
        # Only save the model in in push_saved_epochs
        if epoch in push_saved_epochs:
            bounding_box_tracker = push.push_prototypes(
                train_push_loader, # pytorch dataloader (must be unnormalized in [0,1])
                prototype_network_parallel=ppnet_multi, # pytorch network with prototype_vectors
                class_specific=class_specific,
                preprocess_input_function=preprocess_input_function, # normalize if needed
                prototype_layer_stride=1,
                root_dir_for_saving_prototypes=None, # if not None, prototypes will be saved here
                epoch_number=epoch, # if not provided, prototypes saved previously will be overwritten
                prototype_img_filename_prefix=None,
                prototype_self_act_filename_prefix=None,
                proto_bound_boxes_filename_prefix=None,
                save_prototype_class_identity=True,
                log=log,
                bounding_box_tracker=None)
            accu = tnt.test(model=ppnet_multi, dataloader=test_loader,
                            class_specific=class_specific, log=log)
            save.save_model_w_condition(model=ppnet, model_dir=model_dir, model_name=str(epoch) + 'push', accu=accu,
                                        target_accu=0.76, log=log)

           

        if prototype_activation_function != 'linear':
            tnt.last_only(model=ppnet_multi, log=log)
            # Fine tune the last layers
            for i in range(8):
                log('iteration: \t{0}'.format(i))
                _ = tnt.train(model=ppnet_multi, dataloader=train_loader, optimizer=last_layer_optimizer,
                              class_specific=class_specific, coefs=coefs, log=log)
                accu = tnt.test(model=ppnet_multi, dataloader=test_loader,
                    class_specific=class_specific, log=log)
                print("Test accuracy: ", accu)
                save.save_model_w_condition(model=ppnet, model_dir=model_dir, model_name=str(epoch) + '_' + str(i) + 'push', accu=accu,
                                target_accu=0.76, log=log)
            
            save.save_model_w_condition(model=ppnet, model_dir=model_dir, model_name=str(epoch) + '_' + str(i) + 'push', accu=accu,
                                target_accu=0.76, log=log)
            # Save the last with the final layer fine-tuned
        
    accu = tnt.test(model=ppnet_multi, dataloader=test_loader,
                    class_specific=class_specific, log=log)
    print("Test accuracy: ", accu)
    save.save_model_w_condition(model=ppnet, model_dir=model_dir, model_name=str(epoch), accu=accu,
                                target_accu=0.76, log=log)

epoch: 	0
	warm
	train


1405it [08:39,  2.71it/s]


	time: 	519.3797466754913
	cross ent: 	4.7905920799092465
	cluster: 	1.7534360656216474
	separation:	1.4305785773567246
	avg separation:	9.857729778086163
	accu: 		7.18385051718385%
	l1: 		201000.0
	p dist pair: 	19.467283248901367
	test


46it [00:17,  2.62it/s]


	time: 	17.71340012550354
	cross ent: 	4.0388519038324775
	cluster: 	0.15035044045551962
	separation:	0.02786378910684067
	avg separation:	9.562138951343039
	accu: 		13.030721435968243%
	l1: 		201000.0
	p dist pair: 	19.467283248901367
Test accuracy:  0.13030721435968243
epoch: 	1
	warm
	train


...

	time: 	929.8788900375366
	cross ent: 	0.01822818226422916
	cluster: 	0.01943254183805498
	separation:	25.666421083877943
	avg separation:	37.813054847038515
	accu: 		99.99721944166389%
	l1: 		10302.90234375
	p dist pair: 	37.45183181762695
	test


46it [00:17,  2.68it/s]


	time: 	17.385366678237915
	cross ent: 	1.9813822948414346
	cluster: 	4.820735039918319
	separation:	19.920073529948358
	avg separation:	36.80878394582997
	accu: 		78.82292026234036%
	l1: 		10302.90234375
	p dist pair: 	37.45183181762695
Test accuracy:  0.7882292026234036
	above 76.00%
epoch: 	27
	joint
	train


1353it [14:56,  1.51it/s]


KeyboardInterrupt: 